In [1]:
import pandas as pd
train=pd.read_csv('../../input/final/train_final.csv').drop('Unnamed: 0', axis=1)
train_target=pd.read_csv('../../input/final/train_target_final.csv').drop('Unnamed: 0', axis=1)
test=pd.read_csv('../../input/final/test_final.csv').drop('Unnamed: 0', axis=1)

In [2]:
from xgboost import XGBClassifier

/home/rb/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
xgb1 = XGBClassifier(learning_rate=0.2, seed=42, silent=1)
xgb1.fit(train, train_target)
pred=xgb1.predict_proba(test)

/home/rb/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rb/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
test_pred=pd.concat([test, pd.DataFrame(pred)],axis=1)
test_pred=test_pred[['Date_Ord','Lat','Long', 1]]

In [5]:
from datetime import datetime
test_pred['Date']=[datetime.fromordinal(d) for d in test_pred['Date_Ord']]
test_pred=test_pred.drop('Date_Ord', axis=1)
train=pd.concat([train[['Lat', 'Long', 'Date_Ord']], train_target], axis=1)
train['Date']=[datetime.fromordinal(d) for d in train['Date_Ord']]
train=train.drop('Date_Ord', axis=1)

In [6]:
test_pred=test_pred.drop_duplicates()
test_pred=test_pred[test_pred['Date'].dt.year==2014]

In [7]:
rel=test_pred.sort_values(1, ascending=False).reset_index().drop('index', axis=1)

In [8]:
plans=[]
for p in [_+1 for _ in range(10)]:
    locs=rel.iloc[1:(1107*p)]
    plans.append(locs.drop(1, axis=1).drop_duplicates())

In [9]:
del plans[-1]
cost=[]
for plan in plans:
    cost.append(len(plan)*1100)

In [10]:
new_test=pd.read_csv('../../input/final/test_final.csv').drop('Unnamed: 0', axis=1)

In [11]:
new_test=new_test[new_test['Year']==2014]

In [12]:
def spray_factor(row):
    Date = row.Date
    Lat = row.Lat
    Long = row.Long   
    rel=spray[(spray['Date']<Date) & ((Date-spray['Date'])<pd.Timedelta('14 days'))]
    s=0
    for _ in rel.iterrows():
        latdiff=Lat-_[1]['Lat']
        longdiff=Long-_[1]['Long']
        dis=.5**(latdiff**2+longdiff**2)
        t=(Date-_[1]['Date']).days
        s+=(.5)**(t/1.5)/dis
    return s

In [13]:
spray_factors=[]

In [14]:
new_test_locs=new_test[['Lat', 'Long', 'Date_Ord']]
new_test_locs['Date']=[datetime.fromordinal(d) for d in new_test_locs['Date_Ord']]
new_test_locs=new_test_locs.drop('Date_Ord', axis=1)

/home/rb/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
plan_spray_factors=[]
for i in range(10):
    spray=plans[i]
    plan_spray_factors.append([spray_factor(row[1]) for row in new_test_locs.iterrows()])
    print i

0
1
2
3
4
5
6
7
8


IndexError: list index out of range

In [67]:
spray_preds=[]
for i in range(9):
    new_test['Spray_Factor']=plan_spray_factors[i]
    spray_preds.append(xgb1.predict_proba(new_test))

In [68]:
adj_preds=[]
for i in range(9):
    adj_preds=[_[1] for _ in spray_preds[i]]

In [69]:
sum(pred)

1251.9241873748906

In [74]:
spray_preds[0]

array([[  9.96163368e-01,   3.83663666e-03],
       [  9.97708917e-01,   2.29110895e-03],
       [  9.97538328e-01,   2.46169535e-03],
       ..., 
       [  9.99239802e-01,   7.60205381e-04],
       [  9.94293272e-01,   5.70672145e-03],
       [  9.99239802e-01,   7.60205381e-04]], dtype=float32)

In [75]:
spray_preds[1]

array([[  9.96163368e-01,   3.83663666e-03],
       [  9.97708917e-01,   2.29110895e-03],
       [  9.97538328e-01,   2.46169535e-03],
       ..., 
       [  9.99239802e-01,   7.60205381e-04],
       [  9.94293272e-01,   5.70672145e-03],
       [  9.99239802e-01,   7.60205381e-04]], dtype=float32)

In [77]:
sum(pred)

1251.9241873748906

,Lat,Long,WnvPresent,Date
0,41.867108,-87.654224,0,2007-05-29
1,41.867108,-87.654224,0,2007-05-29
2,41.862292,-87.648860,0,2007-05-29
3,41.896282,-87.655232,0,2007-05-29
4,41.907645,-87.760886,0,2007-05-29
5,41.921600,-87.666455,0,2007-05-29
6,41.720848,-87.666014,0,2007-05-29
7,41.919343,-87.694259,0,2007-05-29
8,41.921965,-87.632085,0,2007-05-29
9,41.921965,-87.632085,0,2007-05-29
